In [ ]:
import akshare as ak
import pandas as pd

In [2]:
def and_filter_fund(data_source: pd.DataFrame, top_number: int) -> pd.DataFrame:
    # top_month_codes = set(data_source.nlargest(top_number, "近1月")["基金代码"])
    top_3months_codes = set(data_source.nlargest(top_number, "近3月")["基金代码"])
    top_6months_codes = set(data_source.nlargest(top_number, "近6月")["基金代码"])
    top_year_codes = set(data_source.nlargest(top_number, "近1年")["基金代码"])
    top_2year_codes = set(data_source.nlargest(top_number, "近2年")["基金代码"])
    top_3year_codes = set(data_source.nlargest(top_number, "近3年")["基金代码"])

    # 计算共同基金代码的交集
    common_codes = (
        # top_month_codes &
        top_3months_codes
        & top_year_codes
        & top_6months_codes
        & top_6months_codes
        & top_2year_codes
        & top_3year_codes
    )

    # 筛选出这些共同基金
    return data_source[data_source["基金代码"].isin(common_codes)]

# 开放基金


## Overall ranking


In [3]:
and_filter_fund(ak.fund_open_fund_rank_em(), 200)

,序号,基金代码,基金简称,日期,单位净值,累计净值,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,自定义,手续费
28,29,010768,建信利率债策略纯债债券C,2024-10-15,1.4097,1.4637,0.01,0.38,32.45,33.42,34.63,37.48,39.87,44.23,36.63,48.35,37.4845,0.00%


## 混合型


In [4]:
bond_fund = ak.fund_open_fund_rank_em(symbol="混合型")
and_filter_fund(bond_fund, 150)

,序号,基金代码,基金简称,日期,单位净值,累计净值,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,自定义,手续费
148,149,217021,招商优势企业混合A,2024-10-15,4.1571,4.1571,-3.15,-11.51,30.37,25.23,22.97,18.54,60.63,32.86,18.54,315.71,19.7321,0.15%


## 股票型


In [5]:
bond_fund = ak.fund_open_fund_rank_em(symbol="股票型")
and_filter_fund(bond_fund, 70)

,序号,基金代码,基金简称,日期,单位净值,累计净值,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,自定义,手续费


## 债券型


In [6]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="债券型"), 130)

,序号,基金代码,基金简称,日期,单位净值,累计净值,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,自定义,手续费
5,6,010768,建信利率债策略纯债债券C,2024-10-15,1.4097,1.4637,0.01,0.38,32.45,33.42,34.63,37.48,39.87,44.23,36.63,48.35,37.4845,0.00%
12,13,007540,华泰保兴安悦债券A,2024-10-15,1.0880,1.2363,0.05,1.60,-0.33,3.38,5.38,12.28,14.11,17.38,10.06,25.21,12.2999,0.08%
13,14,006828,银河久泰债券,2024-10-15,1.1028,1.3308,0.09,0.55,8.48,8.68,9.02,11.26,13.39,18.93,10.60,35.87,11.2785,0.80%
18,19,519947,长信利保债券A,2024-10-15,1.1687,1.1687,-0.01,-0.07,6.16,6.11,6.63,9.45,9.62,13.92,8.51,16.87,9.4084,0.08%
19,20,008176,长信利保债券C,2024-10-15,1.1686,1.1686,-0.01,-0.07,6.16,6.11,6.63,9.44,9.59,13.88,8.52,18.48,9.4092,0.00%


## 指数型

In [7]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="指数型"), 80)

,序号,基金代码,基金简称,日期,单位净值,累计净值,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,自定义,手续费


# 货币基金


In [8]:
and_filter_fund(ak.fund_money_rank_em(), 30)

,序号,基金代码,基金简称,日期,万份收益,年化收益率7日,年化收益率14日,年化收益率28日,近1月,近3月,近6月,近1年,近2年,近3年,近5年,今年来,成立来,手续费
6,7,001821,兴全天添益货币B,2024-10-15,0.5283,1.919,1.9095,1.9273,0.16,0.47,0.97,2.15,4.49,6.90,12.59,1.65,29.06,0.00%
8,9,004863,泰康现金管家货币C,2024-10-15,0.4036,1.838,1.9309,1.8592,0.15,0.47,0.98,2.15,4.44,6.67,11.69,1.66,19.99,0.00%
9,10,004862,泰康现金管家货币B,2024-10-15,0.4035,1.838,1.9310,1.8591,0.15,0.47,0.98,2.15,4.44,6.67,11.69,1.66,19.99,0.00%
10,11,002234,泰信天天收益货币B,2024-10-15,0.4747,1.722,1.8054,1.9147,0.16,0.46,0.95,2.14,4.49,6.90,11.75,1.63,24.26,0.00%
13,14,202308,南方收益宝货币B,2024-10-15,0.5102,1.881,1.8916,1.8865,0.15,0.46,0.97,2.14,4.39,6.74,12.19,1.65,40.39,0.00%
18,19,002680,工银安盈货币B,2024-10-15,0.4973,1.805,1.8285,1.9763,0.16,0.46,0.95,2.13,4.40,6.76,12.12,1.63,26.59,0.00%
19,20,003474,南方天天利货币B,2024-10-15,0.5128,1.906,1.9014,1.8897,0.15,0.46,0.96,2.13,4.37,6.79,12.29,1.65,25.72,0.00%


# 场内交易基金


In [9]:
and_filter_fund(ak.fund_exchange_rank_em(), 50)

,序号,基金代码,基金简称,类型,日期,单位净值,累计净值,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,成立日期
